# Look up structured data with retrieval UDFs

Create lookup functions that query tables by key—for customer records, product catalogs, or financial data.


## Problem

You have structured data—customer records, product catalogs, financial data—and need to look up rows by key values. Common scenarios:

| Use case | Lookup key | Returns |
|----------|------------|---------|
| Customer support | `customer_id` | Contact info, order history |
| Product search | `sku` or `product_name` | Price, inventory, specs |
| Financial analysis | `ticker` or `date` | Stock prices, transactions |
| Inventory check | `warehouse_id`, `product_id` | Stock levels |


## Solution

**What's in this recipe:**
- Create lookup functions from tables with `retrieval_udf`
- Query by single or multiple keys
- Use lookups in computed columns for data enrichment

Use `pxt.retrieval_udf(table)` to automatically create a function that queries the table by its columns.


### Setup


In [ ]:
%pip install -qU pixeltable


In [ ]:
import pixeltable as pxt


In [ ]:
# Create a fresh directory
pxt.drop_dir('lookup_demo', force=True)
pxt.create_dir('lookup_demo')


### Create a product catalog table


In [ ]:
# Create a product catalog
products = pxt.create_table(
    'lookup_demo.products',
    {'sku': pxt.String, 'name': pxt.String, 'price': pxt.Float, 'category': pxt.String}
)

products.insert([
    {'sku': 'LAPTOP-001', 'name': 'MacBook Pro 14"', 'price': 1999.00, 'category': 'electronics'},
    {'sku': 'LAPTOP-002', 'name': 'ThinkPad X1', 'price': 1499.00, 'category': 'electronics'},
    {'sku': 'PHONE-001', 'name': 'iPhone 15 Pro', 'price': 999.00, 'category': 'electronics'},
    {'sku': 'CHAIR-001', 'name': 'Ergonomic Office Chair', 'price': 449.00, 'category': 'furniture'},
    {'sku': 'DESK-001', 'name': 'Standing Desk', 'price': 699.00, 'category': 'furniture'},
])

products.collect()


### Create a lookup function with retrieval_udf


In [ ]:
# Create a lookup function that searches by SKU
get_product = pxt.retrieval_udf(
    products,
    name='get_product',
    description='Look up a product by its SKU code',
    parameters=['sku'],  # Only use SKU as the lookup key
    limit=1  # Return at most 1 result
)


In [ ]:
# Look up a product by SKU
products.select(get_product(sku='LAPTOP-001')).limit(1).collect()


### Look up by category (multiple results)


In [ ]:
# Create a category lookup (returns multiple products)
get_by_category = pxt.retrieval_udf(
    products,
    name='get_by_category',
    description='Get all products in a category',
    parameters=['category'],
    limit=10  # Return up to 10 products
)

# Find all electronics
products.select(get_by_category(category='electronics')).limit(1).collect()


### Use lookups for data enrichment


In [ ]:
# Create an orders table
orders = pxt.create_table(
    'lookup_demo.orders',
    {'order_id': pxt.String, 'product_sku': pxt.String, 'quantity': pxt.Int}
)

orders.insert([
    {'order_id': 'ORD-001', 'product_sku': 'LAPTOP-001', 'quantity': 2},
    {'order_id': 'ORD-002', 'product_sku': 'PHONE-001', 'quantity': 1},
    {'order_id': 'ORD-003', 'product_sku': 'CHAIR-001', 'quantity': 4},
])


In [ ]:
# Add a computed column that enriches orders with product details
orders.add_computed_column(product_info=get_product(sku=orders.product_sku))

# View enriched orders
orders.select(
    orders.order_id,
    orders.product_sku,
    orders.quantity,
    orders.product_info
).collect()


## Explanation

**`retrieval_udf` parameters:**

| Parameter | Description |
|-----------|-------------|
| `table` | The table to query |
| `name` | Function name (defaults to table name) |
| `description` | Description for LLM tool use |
| `parameters` | Columns to use as lookup keys |
| `limit` | Max rows to return (None = all) |

**Use cases:**

| Pattern | Example |
|---------|---------|
| Data enrichment | Join order SKUs to product details |
| LLM tool calling | Let agents query databases |
| Reference lookups | Convert codes to descriptions |
| Cross-table joins | Link related records |

**Tips:**
- Use `limit=1` for unique key lookups
- Specify only needed columns in `parameters` for cleaner APIs
- Add descriptions for LLM tool integration


## See also

- [Use tool calling with LLMs](./llm-tool-calling.ipynb) - Use retrieval UDFs as LLM tools
- [Build a RAG pipeline](./pattern-rag-pipeline.ipynb) - Semantic search with `@pxt.query`
